In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from scipy.spatial.distance import squareform, pdist
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

In [6]:
model_path = "/Users/weijithwimalasiri/Desktop/WARN/NN_weights/WEIGHTS.hdf5"
ppi_file = "/Users/weijithwimalasiri/Desktop/JustForFun/mimic_perform_af_csv 2/ppi_001_data.txt"
rri_file = "/Users/weijithwimalasiri/Desktop/JustForFun/mimic_perform_af_csv 2/rri_001_data.txt"
model = load_model(model_path)

In [3]:
def normalize_to_gray_scale(matrix):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    normalized_matrix = (matrix - min_val) / (max_val - min_val + 1e-8)  # Avoid division by zero
    return (normalized_matrix*255).astype(np.uint8)

def rp_plot(rri, delay, embedding_dim):
    N= len(rri)
    Nrp = N - (embedding_dim - 1) * delay
    embedded_rri = np.array([rri[i: i + embedding_dim * delay: delay] for i in range(Nrp)])
    distances = squareform(pdist(embedded_rri, metric='euclidean'))
    return distances

In [10]:
img_dim = (224, 224)  # Desired image dimensions
for signal_type, file_path in [('ppi', ppi_file), ('rri', rri_file)]:
    # Read data
    with open(file_path, "r") as f:
        signal = [float(line.strip()) for line in f if line.strip()]

    # Create reurrence plot
    distances = rp_plot(signal[0:43], delay=3, embedding_dim=3)
    rp_gray = normalize_to_gray_scale(distances)
    rp_img = resize(rp_gray, img_dim, anti_aliasing=True, preserve_range=True).astype(np.uint8)

    # Prepare input tensor
    input_tensor = np.expand_dims(rp_img, axis=(0, -1))
    pred = model.predict(input_tensor)

    # Output prediction
    print(f"Prediction using {signal_type}: {pred}")
    pred_class = np.argmax(pred)
    confidence = pred[0][pred_class]
    print(f"Predicted class for {signal_type}: {pred_class} with confidence {confidence:.2f}")


1/1 [==============================] - 0s 387ms/step
Prediction using ppi: [[5.139597e-18 9.999981e-01 1.965519e-06]]
Predicted class for ppi: 1 with confidence 1.00
1/1 [==============================] - 0s 63ms/step
Prediction using rri: [[9.1697423e-09 1.0000000e+00 2.2892317e-08]]
Predicted class for rri: 1 with confidence 1.00
